# batchnorm

批量归一化（Batch Normalization, 简称 Batch Norm 或 BN）是深度学习中常用的一种技术，它用以减少内部协变量偏移（Internal Covariate Shift），加速模型学习过程，同时还能起到一定的正则化效果。

Batch Norm 在训练（Training）和预测（Inference）的时候，处理统计值的方式是不同的。下面分别描述两个阶段的处理方式：

## 训练过程

在训练阶段，Batch Norm 每次处理一个 mini-batch 的数据。它首先计算当前 mini-batch 的均值（mean）和方差（variance），然后使用这些统计值来归一化当前的 mini-batch 数据。

设有输入数据 $X = \{x_1, x_2, ..., x_m\}$，其中 $m$ 是当前 mini-batch 的大小，Batch Norm 计算均值 ($\mu_B$) 和方差 ($\sigma_B^2$) 如下：

$$
\mu_B = \frac{1}{m} \sum_{i=1}^{m} x_i
$$
$$
\sigma_B^2 = \frac{1}{m} \sum_{i=1}^{m} (x_i - \mu_B)^2
$$

然后使用这两个值对输入 $X$ 进行归一化：

$$
\hat{X} = \frac{X - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}
$$

其中的 $\epsilon$ 是一个很小的数（例如 1e-5），以确保分母不为零，是为了数值稳定性考虑。

归一化之后，Batch Norm 还提供了两个可学习参数，缩放因子（γ, scale）和位移（β, shift），用以恢复到可能的最佳的分布状态：

$$
Y = \gamma \hat{X} + \beta
$$

这里的 $Y$ 就是 Batch Norm 层的输出。在训练阶段，γ 和 β 是通过反向传播与其他模型参数一同训练的。

## 预测过程（Inference）

在预测（或测试）阶段，模型会处理可能非常不同的数据，并且通常是一个一个样本地处理，而不是 mini-batch。如果继续使用 mini-batch 的统计值去归一化，那么会因为每次的 mini-batch 不同导致归一化的不稳定，甚至不可能计算（当只有一个数据点时）。

因此，在预测时，Batch Norm 使用在训练过程中累积（通常是指数加权移动平均）的均值和方差的估计：

$$
\mu = E[\mu_B]
$$
$$
\sigma^2 = E[\sigma_B^2]
$$

这里的 $E[\cdot]$ 表示估计值。这些值是在训练过程中计算并更新的，不再随 mini-batch 的不同而变化。

预测时的 Batch Norm 归一化过程如下：

$$
\hat{x}_{\text{test}} = \frac{x_{\text{test}} - \mu}{\sqrt{\sigma^2 + \epsilon}}
$$

对应的输出是：

$$
y_{\text{test}} = \gamma \hat{x}_{\text{test}} + \beta
$$

其中 $x_{\text{test}}$ 是单个测试样本，$y_{\text{test}}$ 是该样本通过 Batch Norm 层的输出。

通过以上的操作，Batch Norm 使得模型在预测时能够使用在训练过程中学到的、更为平稳的数据分布，提高泛化能力。

In [9]:
import torch
import torch.nn as nn
# With Learnable Parameters
m = nn.BatchNorm1d(100)
# Without Learnable Parameters
# m = nn.BatchNorm1d(100, affine=False)
input = torch.randn(20, 100)
output = m(input)

In [11]:
# 打印模型的所有具名参数
for name, param in m.named_parameters():
    print(f"Name: {name},\tShape: {param.size()},\tRequires grad: {param.requires_grad}")

Name: weight,	Shape: torch.Size([100]),	Requires grad: True
Name: bias,	Shape: torch.Size([100]),	Requires grad: True


In [12]:
m

BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [14]:
m.running_mean, m.running_var

(tensor([ 2.2526e-02,  8.3788e-03,  2.5479e-02,  2.0515e-02, -5.3198e-02,
          1.0445e-02, -8.7572e-03,  1.1553e-02,  4.9583e-03, -2.7973e-02,
         -3.1462e-02,  2.2348e-02,  3.2692e-02, -2.1632e-02, -4.3830e-02,
         -2.5293e-02,  6.9338e-03,  2.0425e-02,  2.6334e-03,  2.4866e-02,
         -9.4535e-03, -1.7107e-03,  2.1068e-02, -2.2040e-02, -3.5879e-02,
          6.4485e-04, -8.6509e-03, -1.4788e-02,  3.4774e-02,  5.8979e-03,
          6.8799e-03, -2.2506e-02,  2.3639e-02, -1.1672e-02, -2.2402e-02,
          2.3265e-02, -4.0368e-02, -9.8057e-03,  3.3580e-03, -1.2860e-02,
          5.7512e-03, -2.2749e-02, -7.0760e-03, -1.2083e-02, -1.8619e-02,
         -2.8901e-02, -1.4613e-02,  8.0565e-03,  1.4769e-02,  6.6605e-03,
         -1.2035e-02, -4.4077e-03,  2.9764e-02, -1.9618e-02, -3.1000e-02,
          1.8648e-02, -9.0123e-03,  4.5604e-03,  3.1216e-02,  1.4658e-02,
         -1.0180e-02,  4.4239e-03,  3.5542e-02,  1.7736e-02, -1.7775e-02,
          2.0883e-02,  7.7922e-03, -7.

In [16]:
m.num_batches_tracked, m.training

(tensor(1), True)